<a href="https://colab.research.google.com/github/son50git/uploadFromColabs_002/blob/master/Human_Pose_Milestone_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# created a GPU instance (Edit → Notebook Settings → GPU).

In [2]:
import scipy.io as sio

In [3]:
# download and read the SVHN dataset (train_32x32.mat and test_32x32.mat)
# http://ufldl.stanford.edu/housenumbers/

!wget http://ufldl.stanford.edu/housenumbers/train_32x32.mat
!wget http://ufldl.stanford.edu/housenumbers/test_32x32.mat
train = sio.loadmat('train_32x32.mat')
test = sio.loadmat('test_32x32.mat')

--2020-11-13 21:40:01--  http://ufldl.stanford.edu/housenumbers/train_32x32.mat
Resolving ufldl.stanford.edu (ufldl.stanford.edu)... 171.64.68.10
Connecting to ufldl.stanford.edu (ufldl.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 182040794 (174M) [text/plain]
Saving to: ‘train_32x32.mat’

train_32x32.mat     100%[===================>] 173.61M  12.3MB/s    in 14s     

2020-11-13 21:40:15 (12.2 MB/s) - ‘train_32x32.mat’ saved [182040794/182040794]

--2020-11-13 21:40:15--  http://ufldl.stanford.edu/housenumbers/test_32x32.mat
Resolving ufldl.stanford.edu (ufldl.stanford.edu)... 171.64.68.10
Connecting to ufldl.stanford.edu (ufldl.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64275384 (61M) [text/plain]
Saving to: ‘test_32x32.mat’

test_32x32.mat      100%[===================>]  61.30M  14.8MB/s    in 5.0s    

2020-11-13 21:40:21 (12.2 MB/s) - ‘test_32x32.mat’ saved [64275384/64

In [4]:
## display a few example images and their label

# type(train) # dict
# train.shape # ERROR
# len(train) # 5

train.items()
# train.keys()
# train.values()


dict_items([('__header__', b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Mon Dec  5 21:09:26 2011'), ('__version__', '1.0'), ('__globals__', []), ('X', array([[[[ 33,  84,  19, ...,  92, 190, 216],
         [ 30,  76,  54, ...,  78, 188, 217],
         [ 38,  59, 110, ..., 101, 191, 212]],

        [[ 15,  86,  20, ...,  94, 205, 221],
         [ 23,  73,  52, ...,  82, 203, 222],
         [ 19,  66, 111, ..., 105, 206, 217]],

        [[ 15,  77,  25, ..., 114, 220, 226],
         [ 17,  78,  57, ..., 101, 218, 227],
         [ 19,  56, 116, ..., 125, 220, 221]],

        ...,

        [[ 72,  90,  65, ..., 200, 229, 200],
         [ 65,  78, 144, ..., 201, 231, 199],
         [ 56,  69, 223, ..., 203, 224, 191]],

        [[ 82,  88,  78, ..., 192, 229, 193],
         [ 77,  77, 148, ..., 193, 229, 188],
         [ 57,  67, 218, ..., 195, 224, 182]],

        [[ 89,  88,  98, ..., 190, 229, 197],
         [ 79,  78, 158, ..., 191, 228, 189],
         [ 59,  66, 220, ..., 193,

In [7]:
! pip install boto3 # one-time (??)

# Successfully built boto3
# ERROR: requests 2.23.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.2 which is incompatible.
# ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.

# Installing collected packages: jmespath, urllib3, botocore, s3transfer, boto3
# Found existing installation: urllib3 1.24.3
# Uninstalling urllib3-1.24.3:
# Successfully uninstalled urllib3-1.24.3
# Successfully installed boto3-1.16.17 botocore-1.19.17 jmespath-0.10.0 s3transfer-0.3.3 urllib3-1.26.2
# WARNING: The following packages were previously imported in this runtime:
# [urllib3]
# You must restart the runtime in order to use newly installed versions.

     |████████████████████████████████| 133kB 11.2MB/s 
     |████████████████████████████████| 6.8MB 11.6MB/s 
     |████████████████████████████████| 71kB 12.3MB/s 
     |████████████████████████████████| 143kB 60.1MB/s 
ERROR: requests 2.23.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.2 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [8]:
# This code downloads the coco dataset from Amazon S3 in parallel.
import boto3
from botocore import UNSIGNED
from botocore.client import Config
import multiprocessing
import subprocess

In [9]:

s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

In [10]:
def download_and_unzip_from_s3(file_name, bucket_name='fast-ai-coco'):
    print("Downloading", file_name)
    s3.download_file(bucket_name, file_name, file_name)
    print("Finished downloading", file_name, ". Starting to unzip.")
    subprocess.run(["unzip", file_name])
    print("Finished unzipping", file_name)

In [11]:
num_cpus = multiprocessing.cpu_count()
num_cpus # 2

2

In [12]:
# Download in parallel
# In sections 1 and 3, you don’t need to download the “train2017.zip” file
# since it’s very large. We only need that one in Milestone 4.

## code
# files = ['val2017.zip', 'annotations_trainval2017.zip', 'train2017.zip']
files = ['val2017.zip', 'annotations_trainval2017.zip']
with multiprocessing.Pool(num_cpus) as p:
  p.map(download_and_unzip_from_s3, files)
  
print("Done transferring all datasets")

## outputs
# Downloading annotations_trainval2017.zip
# Downloading val2017.zip
# Finished downloading annotations_trainval2017.zip . Starting to unzip.
# Finished unzipping annotations_trainval2017.zip
# Finished downloading val2017.zip . Starting to unzip.
# Finished unzipping val2017.zip
# Done transferring all datasets

Finished downloading annotations_trainval2017.zip . Starting to unzip.
Finished downloading val2017.zip . Starting to unzip.
Finished unzipping annotations_trainval2017.zip
Finished unzipping val2017.zip
Done transferring all datasets
